# Automated ML

Import Dependencies.

In [1]:
from azureml.core import Workspace, Experiment,Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

import azureml.core
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

import os
import joblib
import pandas as pd
import numpy as np

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-132749
aml-quickstarts-132749
southcentralus
976ee174-3882-4721-b90a-b5fef6b72f24


## Create an Azure ML experiment

In [3]:
experiment_name = 'automl-experiment'
project_folder = './automl-model'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-experiment,quick-starts-ws-132749,Link to Azure Machine Learning studio,Link to Documentation


## Dataset

### Overview

We will build prediction model that will classify if a customer is going to churn or not from kaggle's data set [Credit Card customers](https://www.kaggle.com/sakshigoyal7/credit-card-customers). Data set was downloaded as a csv and uploaded i.e. registered as data set in the workspace.

In [4]:
dataset=Dataset.get_by_name(ws,name="Credit-Card-Churners")
run = experiment.start_logging()
df = dataset.to_pandas_dataframe()
df.describe()

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
count,1.012700e+04,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,7.391776e+08,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,0.759941,4404.086304,64.858695,0.712222,0.274894,0.159997,0.840003
std,3.690378e+07,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,0.219207,3397.129254,23.472570,0.238086,0.275691,0.365301,0.365301
min,7.080821e+08,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000,0.000008,0.000420
25%,7.130368e+08,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,0.631000,2155.500000,45.000000,0.582000,0.023000,0.000099,0.999660
50%,7.179264e+08,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,0.736000,3899.000000,67.000000,0.702000,0.176000,0.000181,0.999820
75%,7.731435e+08,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,0.859000,4741.000000,81.000000,0.818000,0.503000,0.000337,0.999900
max,8.283431e+08,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000,0.999580,0.999990


In [5]:
currDir=os.getcwd()
print(currDir)
os.listdir(currDir)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-aml-cluster/code/Users


['.config',
 '.ipynb_checkpoints',
 'automl(1).ipynb',
 'automl(2).ipynb',
 'automl.ipynb',
 'conda_dependencies.yml',
 'CreditCardChurners.csv',
 'hyperparameter_tuning new ScriptRunConfig.ipynb',
 'hyperparameter_tuning.ipynb',
 'odl_user_132749',
 'outputs',
 'README.md',
 'train.py',
 'udacity-project.ipynb']

## Create or Attach a Compute Resource

In [6]:
compute_name = "compute-aml-cluster"

# Check if the compute target exists
try:
    compute_aml_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisionibng_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=5)
    compute_aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)

compute_aml_cluster.wait_for_completion(show_output=True)

Found existing cluster.

Running


In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Use the clean_data function to clean your data.
#x, y = clean_data(df)

def binary_encode(df, column, positive_value):
    df = df.copy()
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def ordinal_encode(df, column, ordering):
    df = df.copy()
    df[column] = df[column].apply(lambda x: ordering.index(x))
    return df

def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

def pre_processing(data):
    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()

    # Drop last two columns (unneeded)
    x_df.drop(x_df.columns[-2:],inplace=True, axis=1)

    # Drop CLIENTNUM columns
    x_df.drop("CLIENTNUM",inplace=True, axis=1)

    # Encode unknown values as np.NaN
    x_df = x_df.replace('Unknown', np.NaN)

    # Fill ordinal missing values with modes (Education_Level and Income_Category columns)
    x_df['Education_Level'] = x_df['Education_Level'].fillna('Graduate')
    x_df['Income_Category'] = x_df['Income_Category'].fillna('Less than $40K')

    # Encode binary columns
    x_df = binary_encode(x_df, 'Attrition_Flag', positive_value='Attrited Customer')
    x_df = binary_encode(x_df, 'Gender', positive_value='M')

    # Encode ordinal columns
    education_ordering = [
        'Uneducated',
        'High School',
        'College',
        'Graduate',
        'Post-Graduate',
        'Doctorate'
    ]
    income_ordering = [
        'Less than $40K',
        '$40K - $60K',
        '$60K - $80K',
        '$80K - $120K',
        '$120K +'
    ]

    x_df = ordinal_encode(x_df, 'Education_Level', ordering=education_ordering)
    x_df = ordinal_encode(x_df, 'Income_Category', ordering=income_ordering)

    # Encode nominal columns
    x_df = onehot_encode(x_df, 'Marital_Status', prefix='Marital_Status')
    x_df = onehot_encode(x_df, 'Card_Category', prefix='Card_Category')

    # Split df into X and y
    X = x_df.drop('Attrition_Flag', axis=1).copy()
    y = x_df['Attrition_Flag'].copy()

    # Scale X with a standard scaler
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    return X, y

x, y = pre_processing(dataset)

all_data = pd.concat([x,y], axis = 1)
all_data.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Income_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,...,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver,Attrition_Flag
0,-0.165406,1.059956,0.503368,-0.893680,0.597300,0.384621,0.763943,-1.327136,0.492404,0.446622,...,3.834003,-0.775882,-0.282405,1.077338,-0.798507,0.270611,-0.107644,-0.044484,-0.240794,0
1,0.333570,-0.943436,2.043199,0.593388,-0.887628,1.010715,1.407306,-1.327136,-0.411616,-0.041367,...,12.608573,-0.616276,-0.282405,-0.928214,1.252337,0.270611,-0.107644,-0.044484,-0.240794,0
2,0.583058,1.059956,0.503368,0.593388,1.339764,0.008965,0.120579,-1.327136,-2.219655,-0.573698,...,6.807864,-0.997155,-0.282405,1.077338,-0.798507,0.270611,-0.107644,-0.044484,-0.240794,0
3,-0.789126,-0.943436,1.273283,-0.893680,-0.887628,-0.241473,-0.522785,1.641478,-1.315636,-0.585251,...,6.807864,1.759686,-0.282405,-0.928214,-0.798507,0.270611,-0.107644,-0.044484,-0.240794,0
4,-0.789126,1.059956,0.503368,-1.637214,0.597300,-1.869317,0.763943,-1.327136,-2.219655,-0.430877,...,7.509325,-0.997155,-0.282405,1.077338,-0.798507,0.270611,-0.107644,-0.044484,-0.240794,0


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(all_data, test_size=0.2,random_state=0)
# columns=x_train.columns
print(x_train.head(3))
print(x_test.head(3))
print(x_train.shape)
print(x_test.shape)

if "training" not in os.listdir():
    os.mkdir("./training")

x_train.to_csv('training/train_data.csv')
x_test.to_csv('training/test_data.csv')

# x_train.to_csv(path_or_buf='training/train_data.csv', columns=columns, header=True, index=False) 
# x_test.to_csv(path_or_buf='training/test_data.csv', columns=columns, header=True, index=False)

      Customer_Age    Gender  Dependent_count  Education_Level  \
147       1.082034  1.059956        -0.266547         0.593388   
1985     -1.787077  1.059956        -1.806378         0.593388   
8316     -1.412845 -0.943436         1.273283        -1.637214   

      Income_Category  Months_on_book  Total_Relationship_Count  \
147          2.082228        1.762028                 -0.522785   
1985        -0.887628       -1.118005                  1.407306   
8316        -0.887628       -1.368442                 -1.166148   

      Months_Inactive_12_mon  Contacts_Count_12_mon  Credit_Limit  ...  \
147                -0.337598              -2.219655      0.251316  ...   
1985                0.651940               0.492404      0.066684  ...   
8316                0.651940              -0.411616     -0.577989  ...   

      Total_Ct_Chng_Q4_Q1  Avg_Utilization_Ratio  Marital_Status_Divorced  \
147              1.208772              -0.485688                -0.282405   
1985           

In [9]:
# get the datastore to upload prepared data
datastore = ws.get_default_datastore()
print(datastore)

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='training/', target_path='data/')
# datastore.upload_files(['trainset.csv'])

# create a dataset referencing the cloud location
train_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ('data/train_data.csv'))])

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-c1da3321-82da-4cff-8885-886668087e61",
  "account_name": "mlstrg132749",
  "protocol": "https",
  "endpoint": "core.windows.net"
}
Uploading an estimated of 2 files
Target already exists. Skipping upload for data/test_data.csv
Target already exists. Skipping upload for data/train_data.csv
Uploaded 0 files


## AutoML Configuration

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 3,
    "max_cores_per_iteration":-1
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_aml_cluster,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="Attrition_Flag",   
                             path = project_folder,
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings
)

## Submit Run

In [11]:
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on compute-aml-cluster with default configuration
Running on remote compute: compute-aml-cluster
Parent Run ID: AutoML_c1e7d13b-87cd-4132-b764-99497c57aaeb

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+--------------------

## Run Details

In [12]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_c1e7d13b-87cd-4132-b764-99497c57aaeb,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|1318                             |1                                |8101                                  |
+---------------------------------+----------

{'runId': 'AutoML_c1e7d13b-87cd-4132-b764-99497c57aaeb',
 'target': 'compute-aml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-01T12:09:49.222377Z',
 'endTimeUtc': '2021-01-01T12:25:28.36566Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'compute-aml-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-132749","workspace_name":"quick-starts-ws-132749","region":"southcentralus","compute_target":"compute-aml-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max"

## Best Model

In [14]:
best_run, fitted_automl_best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
print('Best Run ID',best_run.id)

Best Run ID AutoML_c1e7d13b-87cd-4132-b764-99497c57aaeb_48


In [15]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_c1e7d13b-87cd-4132-b764-99497c57aaeb_48,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
fitted_automl_best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.10368421052631578,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=600,
     

In [17]:
# Metric in best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print('\nAccuracy of Best run',best_run_metrics['accuracy'],sep='\n')
print(best_run)

model_name = best_run.properties['model_name']
model_name

AUC_micro 0.9988322071151939
matthews_correlation 0.9576683612324097
average_precision_score_macro 0.9950811047722569
balanced_accuracy 0.975378526769395
average_precision_score_micro 0.9988524336987495
AUC_macro 0.9980359673001894
recall_score_micro 0.9885199354605748
f1_score_weighted 0.9884787521575996
weighted_accuracy 0.9934225703198244
precision_score_macro 0.9823221104805429
recall_score_weighted 0.9885199354605748
AUC_weighted 0.9980359673001895
recall_score_macro 0.975378526769395
average_precision_score_weighted 0.9981344883921791
precision_score_micro 0.9885199354605748
precision_score_weighted 0.9884748441475115
accuracy 0.9885199354605748
f1_score_micro 0.9885199354605748
log_loss 0.08394512918228525
norm_macro_recall 0.9507570535387898
f1_score_macro 0.9787994149473626
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_c1e7d13b-87cd-4132-b764-99497c57aaeb_48/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_c1e7d13b-87cd-4132-b764-99497c5

'AutoMLc1e7d13b848'

In [18]:
#TODO: Save the best model

os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_automl_best_model, filename='outputs/automl.joblib')

['outputs/automl.joblib']

## Fetch Environment

In [ ]:
env = best_run.get_environment()

script_file = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#Register the model
model = remote_run.register_model(model_name=model_name)
print(model.name, model.id, model.version, sep='\t')

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file, environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

deploy_service_name= 'automl-model-deployment'
service = Model.deploy(ws,deploy_service_name,  [model], inference_config, deployment_config)

service.wait_for_deployment(show_output = True)

print("State: ",service.state)
print("\nScoring URI: ",service.scoring_uri)
print("\nService Logs",service.get_logs())

## Test the Deploy model
TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service